### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
sd = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
numstu = sd['Student ID'].count()
sd_grouped = sd.groupby(['school_name'])
school_average = sd_grouped.mean()
numschl = sd['school_name'].nunique()
rdscr = school_average['reading_score'].mean()
psrd = sd['Student ID'].loc[sd['reading_score']>=70].count()
mthscr = school_average['math_score'].mean()
psmth = sd['Student ID'].loc[sd['math_score']>=70].count()
pctrd = round(psrd/numstu,3) * 100
pctmth = round(psmth/numstu,3) * 100
pctpas = round((pctrd + pctmth)/2)
totbudg = school_data['budget'].sum()

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [ ]:
ds=pd.DataFrame( {'Total Schools':numschl, 'Total Students':numstu,'Total Budget':totbudg, 
       'Math:Average Score':mthscr, 'Math:Percent Passed':pctmth,
      'Reading: Average Score':[rdscr], 'Reading:Percent Passed':pctrd, 
     'Percent Passed Avg': pctpas})
ds['Total Budget']=ds['Total Budget'].map("${:,.2f}".format)
ds['Total Students']=ds['Total Students'].map("{:,}".format)
ds = ds.set_index('Total Schools')
ds.head()



## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [ ]:
del school_average['Student ID']
del school_average['School ID']
del school_average['size']
del school_average['budget']
del school_data['School ID']
ss = pd.merge(school_data, school_average, how="left", on=["school_name", "school_name"])
psmth = sd.loc[sd['math_score']>=70]
psmth_byschool = psmth.groupby(['school_name'])['math_score'].count()
psrd = sd.loc[sd['reading_score']>=70]
psrd = psrd.groupby(['school_name'])['reading_score'].count()
ss['Per Student Budget']=ss['budget']/ss['size']
ss = pd.merge(ss, psrd, how="left", on=["school_name", "school_name"])
ss = pd.merge(ss, psmth_byschool, how="left", on=["school_name", "school_name"])
ss['% Passing Math']= ss['math_score_y']/ss['size']*100
ss['% Passing Reading'] = ss['reading_score_y']/ss['size']*100
ss['% Overall Passing Rate'] = (ss['% Passing Math']+ss['% Passing Reading'])/2
ss = ss.rename(columns={'type':'School Type','size':'Total Students', 'budget':'Total School Budget',
    'reading_score_x':'Average Reading Score',  'math_score_x':'Average Math Score'})
del ss['reading_score_y']
del ss['math_score_y']
ss = ss [['school_name','School Type', 'Total Students', 'Total School Budget', 'Per Student Budget', 
          'Average Math Score', 'Average Reading Score', '% Passing Math', '% Passing Reading',
         '% Overall Passing Rate']]
ss.head()

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [ ]:
top_performing= ss.sort_values('% Overall Passing Rate', ascending = False)
top_performing['Total School Budget']=top_performing['Total School Budget'].map("${:,.2f}".format)
top_performing['Per Student Budget']=top_performing['Per Student Budget'].map("${:.2f}".format)
top_performing=top_performing.set_index('school_name')
top_performing=top_performing.rename_axis(None)
top_performing.head(5)

## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [ ]:
bottom_performing = ss.sort_values('% Overall Passing Rate')
bottom_performing['Total School Budget']=bottom_performing['Total School Budget'].map("${:,.2f}".format)
bottom_performing['Per Student Budget']=bottom_performing['Per Student Budget'].map("${:.2f}".format)
bottom_performing=bottom_performing.set_index('school_name')
bottom_performing=bottom_performing.rename_axis(None)
bottom_performing.head(5)

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [ ]:
sd=sd.rename(columns={'school_name':'School Name','grade':' '})
sd=sd.replace('9th', ' 9th')
avg_bygrade = sd.groupby(['School Name',' ']).mean()
del avg_bygrade['School ID']
del avg_bygrade['Student ID']
del avg_bygrade['size']
del avg_bygrade['budget']

In [ ]:
avg_math = avg_bygrade.copy()
del avg_math['reading_score']
avg_math = avg_math.rename(columns={'math_score':''})
a= avg_math.unstack(1)
a = a.rename_axis(None)
a

## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [ ]:
avg_read = avg_bygrade.copy()
del avg_read['math_score']
avg_read=avg_read.rename(columns={'reading_score':''})
b=avg_read.unstack(1)
b = b.rename_axis(None)
b

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [ ]:
# Sample bins. Feel free to create your own bins.
school_spending = ss.copy()
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]
school_spending["Spending Ranges (Per Student)"] = pd.cut(school_spending["Per Student Budget"], spending_bins, labels=group_names, include_lowest=True)
del school_spending['Total Students']
del school_spending['Total School Budget']
del school_spending['Per Student Budget']
c=school_spending.groupby(['Spending Ranges (Per Student)']).mean()
c.head()

## Scores by School Size

* Perform the same operations as above, based on school size.

In [ ]:
# Sample bins. Feel free to create your own bins.
school_size = ss.copy()
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_size["School Size"] = pd.cut(school_size["Total Students"], size_bins, labels=group_names, include_lowest=True)
del school_size['Total Students']
del school_size['Total School Budget']
del school_size['Per Student Budget']
d=school_size.groupby(['School Size']).mean()
d.head()

## Scores by School Type

* Perform the same operations as above, based on school type.

In [ ]:
school_type = ss.groupby(['School Type']).mean()
del school_type['Total Students']
del school_type['Total School Budget']
del school_type['Per Student Budget']
school_type.head()